# Contract Analysis
- We apply the neural network with pre-trained weights on norms contained in contracts.

In [1]:
import pandas as pd
from pandas import DataFrame

results_df = pd.read_csv('../result.tsv', sep='\t')
results_df['norm1'] = results_df['norm1'].astype(str)
results_df['len_norm1'] = results_df['norm1'].apply(lambda row: len(row.split()))
results_df['norm2'] = results_df['norm2'].astype(str)
results_df['len_norm2'] = results_df['norm2'].apply(lambda row: len(row.split()))
len_df = results_df[['len_norm1', 'len_norm2']]
display(results_df.columns)
len_df.describe(percentiles=[.25, .5, .75, .99])

Index(['norm1', 'norm2', 'conf_type', 'conflict', 'majority_class',
       'entailment_prob', 'contradiction_prob', 'neutral_prob',
       'rev_majority_class', 'rev_entailment_prob', 'rev_contradiction_prob',
       'rev_neutral_prob', 'len_norm1', 'len_norm2'],
      dtype='object')

,len_norm1,len_norm2
count,228.000000,228.000000
mean,24.890351,20.456140
std,14.704768,14.425345
min,1.000000,1.000000
25%,17.000000,12.750000
50%,22.000000,18.000000
75%,29.000000,25.250000
99%,81.570000,66.920000
max,153.000000,158.000000


In [2]:
conflicts_df = results_df[results_df['conflict']]
conflicts_df['majority_class'].describe()

count         228
unique          3
top       neutral
freq          122
Name: majority_class, dtype: object

In [5]:
entail_conflict_df = conflicts_df[conflicts_df['majority_class'] == 'entailment']
entail_conflict_df.to_csv('entail_conflicts.tsv', sep='\t')
entail_conflict_df.describe()

,conf_type,entailment_prob,contradiction_prob,neutral_prob,rev_entailment_prob,rev_contradiction_prob,rev_neutral_prob,len_norm1,len_norm2
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,1.850000,0.732501,0.085176,0.182322,0.425541,0.154421,0.420038,23.233333,17.633333
std,1.005493,0.131153,0.101243,0.097780,0.355928,0.264964,0.343093,10.925583,8.415012
min,1.000000,0.481853,0.001351,0.017484,0.003201,0.001067,0.015481,1.000000,3.000000
25%,1.000000,0.641663,0.017080,0.108801,0.047023,0.013299,0.122439,16.000000,12.750000
50%,1.500000,0.726825,0.041410,0.174958,0.396712,0.037482,0.275690,22.000000,15.000000
75%,3.000000,0.820593,0.101313,0.259469,0.786030,0.151876,0.714416,27.000000,23.250000
max,4.000000,0.975468,0.384133,0.406457,0.982970,0.938081,0.995431,75.000000,46.000000


In [6]:
con_conflict_df = conflicts_df[conflicts_df['majority_class'] == 'contradiction']
con_conflict_df.to_csv('contradiction_conflicts.tsv', sep='\t')
con_conflict_df.describe()


,conf_type,entailment_prob,contradiction_prob,neutral_prob,rev_entailment_prob,rev_contradiction_prob,rev_neutral_prob,len_norm1,len_norm2
count,46.000000,46.000000,46.000000,46.000000,46.000000,46.000000,46.000000,46.000000,46.000000
mean,1.760870,0.109525,0.784525,0.105950,0.425232,0.135841,0.438927,19.000000,14.478261
std,0.923394,0.117306,0.167017,0.108072,0.322395,0.248588,0.324820,7.192589,5.935727
min,1.000000,0.001020,0.463421,0.002742,0.000494,0.002311,0.001203,7.000000,5.000000
25%,1.000000,0.018230,0.664794,0.027882,0.159038,0.013271,0.135957,16.000000,9.250000
50%,2.000000,0.060538,0.804010,0.079653,0.374656,0.032033,0.433594,17.000000,17.000000
75%,2.000000,0.169293,0.940957,0.140919,0.749546,0.103149,0.724532,22.000000,18.750000
max,4.000000,0.438962,0.994844,0.463607,0.979951,0.997895,0.995742,44.000000,25.000000
